In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import quandl
%matplotlib inline
import warnings; warnings.simplefilter('ignore') #忽略可能会出现的警告信息，警告并不是错误，可以忽略；

paper = pd.read_csv('data/paper_data.csv', sep=' ', parse_dates=True) #读取论文数据，由于这些数据是以空格分隔的加入sep=' '参数能够将空格分隔改为逗号分隔，可以看https://www.nature.com/articles/srep01684看看他的数据标准化思路
data = pd.DataFrame({'Google_week': paper['Google End Date'], 'Debt': paper['debt'].astype(np.float64), 'Date': paper['DJIA Date'], 'DJClose': paper['DJIA Closing Price'].astype(np.float64)}) #重新构建一个新的DataFrame
data['Date'] = pd.to_datetime(data['Date'])
data['Google_week'] = pd.to_datetime(data['Google_week'])

trends_download = pd.read_csv('data/debt_google_trend.csv') #读取下载的Google搜索指数数据，数据在Google Treads上下载(如果是中文可以用百度搜索指数)
trends_download['Week'] = trends_download['Week'].apply(lambda x: pd.to_datetime(x.split(' ')[-1])) #将week列的字符串按空格拆为列表并取最后一个元素并转换为DatetimeIndex并赋值给Week列

all_data = pd.merge(data, trends_download, left_on='Google_week', right_on='Week') #将从两个渠道得到的数据拼接起来(论文的数据进行了标准化，效果比直接调取的好，这是做个比较)
all_data.drop('Week', inplace=True, axis=1)
all_data.set_index('Date', inplace=True)
all_data.rename(columns = {'Debt':'Debt_paper', 'debt':'Debt_download'}, inplace = True)
both_trends = all_data[['Google_week', 'Debt_paper', 'Debt_download']].set_index('Google_week')
both_trends.corr() #分析论文的数据和直接调取的数据的相关性以判断两个数据的差别是否仅有中性化

all_data = all_data.reset_index().set_index('Google_week')
all_data['MA_p'] = all_data['Debt_paper'].shift(1).rolling(window = 3).mean() #往后shift(1)并求三周的移动平均搜索量(站在当周求之前三周的移动平均)
all_data['MA_d'] = all_data['Debt_download'].shift(1).rolling(window = 3).mean()

all_data['signal_p']  = np.where(all_data['Debt_paper'] > all_data['MA_p'], -1, 1) #当本周的搜索指数高于过去三周搜索指数的平均则在下周开多仓
all_data['signal_d']  = np.where(all_data['Debt_download'] > all_data['MA_d'], -1, 1)
all_data.loc[:3, ['signal_p','signal_d']] = 0 #前三个数据因为是当周与NaN空值比较，也会被记为1，所以要改成0

all_data['pct_change'] = all_data['DJClose'].pct_change()
all_data['ret_p'] = all_data['pct_change'] * all_data['signal_p'].shift(1) #第五周进行买卖，计算收益
all_data['ret_d'] = all_data['pct_change'] * all_data['signal_d'].shift(1)

all_data['cumret_p'] = (1 + all_data.ret_p).cumprod() #计算累积收益
all_data['cumret_d'] = (1 + all_data.ret_d).cumprod()

all_data[['cumret_p', 'cumret_d']].plot(figsize=(12,6)) #累计收益作图